<a href="https://colab.research.google.com/github/ahmer-talal/Colab-Files/blob/main/PDC_Paradigms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ahmer Talal   SP23-BCS-041**

Simulating a Fog computing scenario where multiple edge devices collect sensor data (temperature, motion, etc.) and process it in parallel threads before forwarding aggregated results to a simulated cloud server. Use threading or multiprocessing to represent parallelism at the fog layer:

In [1]:
import threading
import random
import time

# ---------------------------
# Simulated Edge Device Class
# ---------------------------
class EdgeDevice(threading.Thread):
    def __init__(self, device_id, sensor_type, results):
        threading.Thread.__init__(self)
        self.device_id = device_id
        self.sensor_type = sensor_type
        self.results = results

    def collect_sensor_data(self):
        if self.sensor_type == "temperature":
            return round(random.uniform(20, 35), 2)
        elif self.sensor_type == "motion":
            return random.choice([0, 1])  # 0=no motion, 1=motion detected
        elif self.sensor_type == "humidity":
            return round(random.uniform(30, 60), 2)

    def process_data(self, value):
        # simple processing: normalization
        return value * 1.1

    def run(self):
        raw_value = self.collect_sensor_data()
        processed = self.process_data(raw_value)

        print(f"[Fog] Device {self.device_id} collected {self.sensor_type}: {raw_value}, processed: {processed}")
        self.results.append(processed)


# ---------------------------
# Cloud Server Function
# ---------------------------
def cloud_server(aggregated_data):
    print("\n--- Sending Data to Cloud ---")
    print(f"Aggregated Fog Layer Data: {aggregated_data}")
    print("Cloud Processing Complete!\n")


# ---------------------------
# MAIN FOG COMPUTING SIMULATION
# ---------------------------
if __name__ == "__main__":
    results = []

    devices = [
        EdgeDevice(1, "temperature", results),
        EdgeDevice(2, "motion", results),
        EdgeDevice(3, "humidity", results),
        EdgeDevice(4, "temperature", results)
    ]

    # Run all devices in parallel
    for d in devices:
        d.start()

    for d in devices:
        d.join()  # wait for all threads to finish

    # Fog layer aggregated output
    fog_output = sum(results) / len(results)
    print(f"\n[Fog] Aggregated Result: {fog_output}")

    # Forward to cloud
    cloud_server(fog_output)


[Fog] Device 1 collected temperature: 27.38, processed: 30.118000000000002
[Fog] Device 2 collected motion: 1, processed: 1.1
[Fog] Device 3 collected humidity: 42.99, processed: 47.28900000000001
[Fog] Device 4 collected temperature: 31.65, processed: 34.815

[Fog] Aggregated Result: 28.3305

--- Sending Data to Cloud ---
Aggregated Fog Layer Data: 28.3305
Cloud Processing Complete!



# **Hadoop MapReduce vs Apache Spark**

MapReduce Simulation:

Apache Spark Implementation (PySpark)

In [6]:
from multiprocessing import Pool
import time

def mapper(line):
    temp = int(line.split(",")[0])
    return temp

def reducer(values):
    return sum(values) / len(values)

start = time.time()

with open("weather_data.txt") as f:
    lines = f.readlines()

with Pool() as p:
    mapped = p.map(mapper, lines)

result = reducer(mapped)

end = time.time()

print("MapReduce Average Temperature:", result)
print("Execution Time (MapReduce):", end - start)


MapReduce Average Temperature: 30.395833333333332
Execution Time (MapReduce): 0.022944211959838867


In [4]:
pip install pyspark


In [7]:
from pyspark import SparkContext
import time

sc = SparkContext("local", "WeatherApp")

start = time.time()

rdd = sc.textFile("weather_data.txt")
temps = rdd.map(lambda line: int(line.split(",")[0]))
avg = temps.mean()

end = time.time()

print("Spark Average Temperature:", avg)
print("Execution Time (Spark):", end - start)

sc.stop()


Spark Average Temperature: 30.395833333333336
Execution Time (Spark): 1.6353590488433838


# **Comparison Result:**

In this experiment of 50-row weather dataset, **Hadoop MapReduce**(Python multiprocessing) computed an average **temperature of 30.39°C** with an execution **time of only 0.02 seconds**, showing efficient performance on small data. Apache Spark produced the same average **temperature (30.39°C)** but took **1.63 seconds** due to its ***JVM and RDD initialization overhead.*** This shows that **MapReduce is faster for small datasets**, while Spark is designed to outperform MapReduce on large-scale, in-memory processing tasks.